## SearchEngine Class

In [4]:
# Data Manipulation
import numpy as np
import pandas as pd

class SearchEngine():
    def __init__(self, data):
        '''
        SearchEngine class
        Input: a songs dataset extracted from spotify API
        '''
        self.data = data

    def target_song(self, title='', artist=''):
        '''
        Search engine function for the target song
        Input: song title and/or artist
        Output: self.target dataset song
        '''
        # transform input strings in lowercase
        title = str(title).lower()
        artist = str(artist).lower()
        # filter self.data on the desired song
        if title != '' and artist != '':
            self.target = self.data[self.data[
                'artists'].str.lower().str.contains(artist)]
            self.target = self.target[self.target[
                'name'].str.lower().str.contains(title)]
        elif title != '':
            self.target = self.data[self.data[
                'name'].str.lower().str.contains(title)]
        elif artist != '':
            self.target = self.data[self.data[
                'artists'].str.lower().str.contains(artist)]
        else:
            print('Please select a song title and artist')
        # keep only the first song result
        self.target = self.target.head(1)
        self.artist = self.target["artists"].to_string(
            index=False).strip("['").strip("']")
        self.title = self.target["name"].to_string(index=False)
        print(f'TITLE: {self.title}')
        print(f'ARTIST: {self.artist}')

## Preprocessor Class

In [5]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.utils import shuffle

# Pipeline and Column Transformers
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn import set_config
set_config(display = "diagram")

# Scaling
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

# Package classes
# from music_similarity.search_engine import SearchEngine

class Preprocessor():
    def __init__(self, se):
        '''
        Preprocessor class
        Input: a songs dataset extracted from spotify API
        '''
        self.se = se

    def scale_data(self):
        '''
        Adapting data function
        '''
        # drop non numerical features before scaling
        self.X=self.se.data.drop(columns=['name','artists'])
        self.X_target=self.se.target.drop(columns=['name','artists'])
        # fit and transofrm with MinMaxScaler
        mmscaler = MinMaxScaler().fit(self.X)
        self.X_mmscaled=mmscaler.transform(self.X)
        self.X_target_mmscaled=mmscaler.transform(self.X_target)
        # fit and transofrm with RobustScaler
        roscaler = RobustScaler().fit(self.X)
        self.X_roscaled=roscaler.transform(self.X)
        self.X_target_roscaled=roscaler.transform(self.X_target)

## Playlist Class

In [6]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Unsupervised Learning
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# K-nn modelling
from sklearn.neighbors import NearestNeighbors

# Package classes
# from music_similarity.search_engine import SearchEngine
# from music_similarity.preprocessor import Preprocessor

class Playlist():
    def __init__(self, preprocessor, se):
        '''
        Extractor class
        Input: preprocessor class dataset extracted from spotify API
        '''
        self.preprocessor = preprocessor
        self.se = se
        self.playlist_songs = 10

    def build_model(self):
        '''
        Model builder function
        '''
        # Define the model
        self.model=NearestNeighbors(
            n_neighbors=self.playlist_songs + 1).fit(
            self.preprocessor.X_mmscaled)
        # Extract index and distance of self.playlist_songs+1
        # number of colest songs
        self.distance, self.index=self.model.kneighbors(
            self.preprocessor.X_target_mmscaled,
            n_neighbors=self.playlist_songs + 1)
        # Copy found index rows from the original not scaled dataset
        self.playlist = self.se.data.iloc[self.index[0],:]
        self.playlist['distance'] = self.distance[0]
        # Remove the target song from the list
        self.playlist = self.playlist.tail(self.playlist_songs)
        # Ordering the playlist on distance, ascending order
        self.playlist = self.playlist.sort_values(
            by=['distance'], ascending=True, ignore_index=True)
        # Drop not necessary columns
        self.playlist = self.playlist[['name', 'artists', 'distance']]
        # Strip square brackets from the artists strings
        self.playlist['artists'] = self.playlist['artists'].apply(
            lambda x: x.strip("['").strip("']"))
        # Set starting index from 0 to 1
        self.playlist.index += 1

In [7]:
if 'se' in globals():
    del se
spotify = pd.read_csv('../raw_data/ML_spotify_data.csv')
se = SearchEngine(spotify)
se.target_song("f", "u2")

if 'preprocessor' in globals():
    del preprocessor
preprocessor = Preprocessor(se)
preprocessor.scale_data()

if 'playlist' in globals():
    del playlist
playlist = Playlist(preprocessor, se)
playlist.build_model()

TITLE: A Sort Of Homecoming - Live
ARTIST: U2


/tmp/ipykernel_29067/2661942349.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.playlist['distance'] = self.distance[0]


In [8]:
playlist.playlist

,name,artists,distance
1,You've Got Another Thing Coming - Live from th...,Judas Priest,0.171471
2,Breaking The Silence - Remastered 2003,Queensrÿche,0.223612
3,"It Ain't Me, Babe - Live at LA Forum, Inglewoo...",Bob Dylan,0.232693
4,"Blasphemous Rumours - Live at Rose Bowl, Pasad...",Depeche Mode,0.236184
5,Feats Don't Fail Me Now - Live at Lisner Audit...,Little Feat,0.238395
6,Shoot Shoot - Live / 2008 Remaster,UFO,0.239608
7,"Nutrocker - Live At Newcastle City Hall, 26.3....","Emerson, Lake & Palmer",0.244874
8,Comin' Atcha Live / Truckin' - Live At The Tro...,Tesla,0.245215
9,Battle Angels,Sanctuary,0.269785
10,Baba O'Riley - Live At Shepperton,The Who,0.270837


## API DATA query

In [64]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# TO DO: I'LL TURN THIS INTO A CLASS LATER

class ApiExtractor:
    def __init__(self, se):
        '''
        Extractor class
        Input: preprocessor class dataset extracted from spotify API
        '''
        # credentials need to be exported via the command line
        self.auth_manager = SpotifyClientCredentials()
        self.sp_connection = spotipy.Spotify(auth_manager=self.auth_manager)
        self.search_limit = 10
        self.se = se
    
    def get_track_attrs(self, artist, title):
        # get track attributes
        self.ta_response = self.sp_connection.search(
            q="artist:" + artist + " track:" + title,
            type="track",
            limit=1)

        # parse attributes from track
        self.track = self.ta_response['tracks']['items'][0]
        self.track_name = self.track['name']
        self.track_uri = self.track['uri']
        self.track_popularity = self.track['popularity']
        self.track_explicit = self.track['explicit']
        self.track_artists = [artist['name'] for artist in self.track['artists']]

        # get track features
        self.track_features = self.sp_connection.audio_features(tracks = self.track_uri)[0]

        return [self.track_name, self.track_artists, self.track_popularity,
                self.track_features['danceability'], self.track_features['valence'],
                self.track_features['energy'], self.track_explicit, self.track_features['key'],
                self.track_features['liveness'], self.track_features['loudness'],
                self.track_features['speechiness'], self.track_features['tempo']]
    
    def create_df_songs(self):
        '''
        Function that create self.df_songs dataframe
        with the following features:
        
        - uri 
        - name 
        - artists 
        - popylarity
        - explicit 
        features
        '''
        # extract the list of artists in a np array
        # artists_array from the baseline dataset
        self.artists_array = np.array([])
        for artist in self.se.data.artists.unique():
            self.artists_array = np.append(self.artists_array, artist)
        
        # Test array for debugging purposes
        # self.test_artists_array = np.array(["['Robin Trower', 'Jack Bruce', 'Bill Lordan']",   TEST
        #                                     "['Michael Hedges']"])                             TEST
         
        # find 10 most popular songs for each artist
        # creating features np arrays to store first 5 informations
        self.songs_uri_array = np.array([])
        self.songs_name_array = np.array([])
        self.songs_artists_array = np.array([])
        self.songs_popularity_array = np.array([])
        self.songs_explicit_array = np.array([])
        
        # Test array to not overload API requests
        # for artist in self.test_artists_array:                                                 TEST
        for artist in self.artists_array:
            # Perform API search request
            time.sleep(0.015)
            print(f'{artist} ...')
            # Try the query
            try:
                self.asl_response = self.sp_connection.search(
                                    q="artist:" + artist,
                                    type="track", 
                                    limit=self.search_limit)
            # If there is an exception:
            # jump to the next loop
            except:
                print(f"Oops! {artist} not valid.  Skipped...")
                continue
            
            # iteration over the self.search_limit songs
            for i in range(len(self.asl_response['tracks']['items'])):
                # append uri of the track
                self.songs_uri_array = np.append(
                    self.songs_uri_array, self.asl_response[
                        'tracks']['items'][i]['uri'])
                # append name of the track
                self.songs_name_array = np.append(
                    self.songs_name_array, self.asl_response[
                        'tracks']['items'][i]['name'])
                # append artists of the track
                self.songs_artists_array = np.append(
                    self.songs_artists_array, artist)
                # append popularity of the track
                self.songs_popularity_array = np.append(
                    self.songs_popularity_array, self.asl_response[
                        'tracks']['items'][i]['popularity'])
                # append explicit of the track
                self.songs_explicit_array = np.append(
                    self.songs_explicit_array, self.asl_response[
                        'tracks']['items'][i]['explicit'])

        self.df_songs = pd.DataFrame()
        self.df_songs['uri'] = self.songs_uri_array
        self.df_songs['name'] = self.songs_name_array
        self.df_songs['artists'] = self.songs_artists_array
        self.df_songs['popularity'] = self.songs_popularity_array
        self.df_songs['explicit'] = self.songs_explicit_array
        #self.df_songs.to_csv('../raw_data/songs.csv')
    
    def create_df_audio_analysis(self):
        '''
        Function that create a self.df_analysis dataframe 
        with the following features:
        
        - 12 pitch features
        - 12 timbre features 
        - uri 
        '''
        # From uri array list, query pitch and timbre features
        self.df_analysis = pd.DataFrame()
        self.df_songs_csv = pd.read_csv('../raw_data/songs.csv')
        self.songs_uri_array_csv = self.df_songs_csv['uri']
        # For testing purposes only iterate over the first 3 uri
        #for uri in self.songs_uri_array[:3]:
        #for uri in self.songs_uri_array:
        for uri in self.songs_uri_array_csv:
            time.sleep(0.015)
            print(f'{uri} ...')
            # Perform API audio_analysis request
            try:
                self.track_analysis = self.sp_connection.audio_analysis(track_id = uri)
            except:
                print(f"Oops! {uri} not valid.  Skipped...")
                continue
            # Reset segments arrays before new song analysis
            self.segment_pitch = []
            self.segment_timbre = []
            # Extract pitch and timbre for each segment of the song
            for segment in range(len(self.track_analysis['segments'])):
                self.segment_pitch.append(self.track_analysis['segments'][segment]['pitches'])
                self.segment_timbre.append(self.track_analysis['segments'][segment]['timbre'])
            # Reset df_song_segments dataframe
            self.df_song_segments = pd.DataFrame()
            # create pitch and timbre columns with np arrays
            self.df_song_segments['segment_pitch'] = self.segment_pitch
            self.df_song_segments['segment_timbre'] = self.segment_timbre
            # Split the array elements in different columns
            self.df_split_pitches = pd.DataFrame(self.df_song_segments['segment_pitch'].tolist(), columns=['sp1', 'sp2', 'sp3', 'sp4', 'sp5', 'sp6', 'sp7', 'sp8', 'sp9', 'sp10', 'sp11', 'sp12'])
            self.df_split_timbres = pd.DataFrame(self.df_song_segments['segment_timbre'].tolist(), columns=['tm1', 'tm2', 'tm3', 'tm4', 'tm5', 'tm6', 'tm7', 'tm8', 'tm9', 'tm10', 'tm11', 'tm12'])
            # Add new columns to df_segment
            self.df_song_segments = pd.concat([self.df_song_segments, self.df_split_pitches, self.df_split_timbres], axis=1)
            self.df_song_segments.drop(['segment_pitch', 'segment_timbre'], axis = 1, inplace = True)
            # Transpose mean serie into a dataframe row
            self.df_song_segments = self.df_song_segments.mean().to_frame().T
            # Add uri column to perform the future merge
            self.df_song_segments['uri'] = uri
            self.df_analysis = pd.concat([self.df_analysis, self.df_song_segments])
        #self.df_analysis.to_csv('../raw_data/analysis.csv')
    
    def create_df_audio_features(self):
        '''
        Function that create a self.df_features dataframe 
        with the following features:
        
        - danceability
        - energy
        - key
        - loudness
        - mode
        - speechiness
        - acousticness
        - instrumentalness
        - liveness
        - valence
        - tempo
        - uri
        '''
        self.df_features = pd.DataFrame()
        self.df_songs_csv = pd.read_csv('../raw_data/songs.csv')
        self.songs_uri_array_csv = self.df_songs_csv['uri']
        #for uri in self.songs_uri_array[:2]:
        for uri in self.songs_uri_array_csv:
            time.sleep(0.015)
            print(f'{uri} ...')
            # Perform API audio_features request
            try:
                self.track_features = self.sp_connection.audio_features(tracks = uri)[0]
            except:
                print(f"Oops! {uri} not valid.  Skipped...")
                continue
            self.df_track_features = pd.DataFrame()
            self.df_track_features = pd.DataFrame.from_dict([self.track_features])
            try:
                self.df_track_features.drop(columns=[
                'type',
                'track_href', 
                'analysis_url', 
                'duration_ms', 
                'time_signature', 
                'id'], inplace=True)
            except:
                print(f"Oops! {uri} has no features.  Skipped...")
            self.df_features = pd.concat([self.df_features, self.df_track_features])
        #self.df_features.to_csv('../raw_data/features.csv')
    
    def merge_dataframes(self):
        '''
        Function that merge all the dataframe toghether
        '''
        self.full_data = pd.merge(self.df_songs, self.df_analysis, how='outer', on='uri')
        self.full_data = pd.merge(self.full_data, self.df_features, how='outer', on='uri')
        

In [ ]:
if 'ae' in globals():
    del ae
ae = ApiExtractor(se)
ae.create_df_audio_features()

spotify:track:0zK545STj6P7qbFSpCK9pp ...
spotify:track:6S3JlDAGk3uu3NtZbPnuhS ...
spotify:track:2XKW8CH8nRZH9cF2DNjBHN ...
spotify:track:54qG9hpUjLgkgTJQ9qvB1P ...
spotify:track:5CmGnAY7I7Ggj303HZBbkX ...
spotify:track:0JqDfzUZZi86BoZnx1MPQv ...
spotify:track:0wb21I1NV5uVUolRXIqlJF ...
spotify:track:3ccHstKEStIOANQsaw078p ...
spotify:track:0V8SWgdKA0BWydzwQflZAY ...
spotify:track:1VrIr1toXX6bOSdotodB71 ...
spotify:track:6zSpb8dQRaw0M1dK8PBwQz ...
spotify:track:3gdewACMIVMEWVbyb8O9sY ...
spotify:track:7rglLriMNBPAyuJOMGwi39 ...
spotify:track:1jDJFeK9x3OZboIAHsY9k2 ...
spotify:track:2TVxnKdb3tqe1nhQWwwZCO ...
spotify:track:5Wj1rJnCLpMHdLaxsFtJLs ...
spotify:track:7HW5WIw7ZgZORCzUxv5gW5 ...
spotify:track:38zsOOcu31XbbYj9BIPUF1 ...
spotify:track:6WCeFNVAXUtNczb7lqLiZU ...
spotify:track:4IRHwIZHzlHT1FQpRa5RdE ...
spotify:track:3LQPTJEqOfljGBxmpgUnoC ...
spotify:track:5lEyMg4GNKnMbaarvT4sRd ...
spotify:track:3beYmUcaXJZiSZMPbsMI7p ...
spotify:track:0emHuukZSuaOzOlsAWHj2W ...
spotify:track:29

spotify:track:4IXg5HtwR1KXWRYIzZBdeN ...
spotify:track:0DeT8WBqfwpvDuEkCyJH7V ...
spotify:track:1LdVhEybhMRi990GM8b5Qo ...
Oops! spotify:track:1LdVhEybhMRi990GM8b5Qo has no features.  Skipped...
spotify:track:2GyzcHu6OmpoV1rcIHBJQ5 ...
spotify:track:2GXuLNYlJGRb7SxZM2YlIb ...
spotify:track:7mjSHL2Eb0kAwiKbvNNyD9 ...
spotify:track:476V2d6iA2tWXgQboKmTtA ...
spotify:track:4d0d2llNcQIHmaCgYTrODQ ...
spotify:track:0wEo4mwfs6tXYgCem9m9wP ...
spotify:track:5y8d8RHvxb0KiTa9Nq0xWp ...
spotify:track:340t55txxJWQmTFK7wARZh ...
spotify:track:5p3nvmoaAY03qGmBQApn0w ...
spotify:track:7kTmndsuOxGut8Oy85Mn62 ...
spotify:track:7eiNxeOC8dvYAJWgTOu6xn ...
spotify:track:5zZqoeBLuXCDVdcjggDQdS ...
spotify:track:0MHXrqn909p0LRTPsNsGEi ...
spotify:track:7zrPswSV1yxrill5OyCuaU ...
spotify:track:4XsH9zBWPOCdXoH9ZDdS8r ...
spotify:track:01gDLZsi0j5fWC28FLPNO8 ...
spotify:track:2ouFIPC3O1HNBcVTw1d9AN ...
spotify:track:7gmB6hVMsv0XbjfiTgpATI ...
spotify:track:1Mq4vRRox3bcruJA75M4D9 ...
spotify:track:7J8F8mHNV79W

spotify:track:2HudfMAHpahXeWpgeZkk3r ...
spotify:track:4tSTpnw8UYNlroCEo774tm ...
spotify:track:7AU0zo4WShS29olDPT5DsN ...
spotify:track:6pqWrRF9K2PHpBmmRSIte4 ...
spotify:track:3toxxSU3TcvzgmeR8LLUiX ...
spotify:track:0W7SzJd9YNDiEUxxeKGdTU ...
spotify:track:5UuikgHTxSRFRnC0zXx10i ...
spotify:track:0MYNeFsKpzBHwrJGfWZysV ...
spotify:track:6MBZyLisOWTChcevgFqwVr ...
spotify:track:1JrNAxiq1o57wxuI9aX6KJ ...
spotify:track:1IQAsCKoXsEJ7tdI0dxTqT ...
spotify:track:2hSOrxX3I69A8xth8arEq2 ...
spotify:track:2it4DH8fqcCkSTCmqY5gzk ...
spotify:track:5bSSkaqVo0qsOmtYOcPpp3 ...
spotify:track:7bo2X63U1NpKYMXV1mZFkq ...
spotify:track:3rFAKoUoTUtREl0ob24L4h ...
spotify:track:5Zpy2HR2vYLjlTQVd81DLH ...
spotify:track:1oAr9H6Qw5gaqBJaZeMwS0 ...
spotify:track:5J7aILM0273RGHm7czdLxx ...
spotify:track:3v19aK0OgDXN1M1KAliHV3 ...
spotify:track:1TEvvaq5thKufLYdlwZV6d ...
spotify:track:4jU7QYKyxlsMCmioO6inQ3 ...
spotify:track:56GBItqD18Be9iN7zGqMX5 ...
spotify:track:6raOBJAoz2vcD2xByWJqqG ...
spotify:track:6H

spotify:track:4XDg07TABQlNRJcTDTuWLA ...
spotify:track:7lLIqYkXGKcpEeauufWlZm ...
spotify:track:60nl7K29XwEJtXCpEE57V4 ...
spotify:track:6xncjf74zeGIdP45l0rEUZ ...
spotify:track:4e41evlv5yGoGbxOa5Gmzh ...
spotify:track:5k4dL7PKD3iPaSsjsxp6lt ...
spotify:track:542C7PqR1oEfl63hlhQO4V ...
spotify:track:6d7Jz2np6jtNaZv1DiWfJs ...
spotify:track:6lMNJzDA47MaCZjjKgam5N ...
spotify:track:4NRFiS0pHjTFDjOm1LWTZj ...
spotify:track:5kXmiepRVuKhhz0SxyCVeL ...
spotify:track:0Tbr4K8UQbzvN5YzQTTwbI ...
spotify:track:0wmk0GFmNaox46CP2h5CGR ...
spotify:track:1FXpAf6Ed6vkBc6EkZHtGT ...
spotify:track:3RRKZ8a8SCtiPZ5qdK2Hos ...
spotify:track:7eOGMNm7MBXBg1m02WFYuj ...
spotify:track:5lc9L9FeLBwlJPgEbq9uEw ...
spotify:track:5WJU527RQNyMLuKecjsL8V ...
spotify:track:6SgQIoLn8kpu8J4wfwGWy8 ...
spotify:track:2w7R3CQgvl8PIKJlwtT9Mv ...
spotify:track:1S4igP7OpCUIEC5l75DssU ...
spotify:track:5aDf6IeBqqRBjoFqymYk1P ...
spotify:track:2X6qfjDzhKQA7UqhzyJrQP ...
spotify:track:2nqpN3cSmL0tTlJ724fE2a ...
spotify:track:3J

spotify:track:0mVEatHvxil3ZGJhL8zI4w ...
spotify:track:5fTwUw09NNgCIOkEKu3bGB ...
spotify:track:6LEiisZCK5EVhYB2kM3AFs ...
spotify:track:1phh5V7T5JAaiIww22ARZk ...
spotify:track:7HNFphOjDJpUGiseV5YvPZ ...
spotify:track:4w3tQBXhn5345eUXDGBWZG ...
spotify:track:2SpEHTbUuebeLkgs9QB7Ue ...
spotify:track:6dnco8haegnJYtylV26cBq ...
spotify:track:5fdhThPDe6jQQDqCyWrdAn ...
spotify:track:6H7nDglS6xWpRidbhMwI2L ...
spotify:track:6nRvgo4ySezQKD1D7NIqAu ...
spotify:track:1l2Xmg54EUr3TDzNIaCr1z ...
spotify:track:5oheUEVCpJllnImTs5nNiS ...
spotify:track:27876bQcDNkZhWqvF0zfFN ...
spotify:track:77bNe6jYY8yHdP7orXrz5I ...
spotify:track:2aoo2jlRnM3A0NyLQqMN2f ...
spotify:track:0wJoRiX5K5BxlqZTolB2LD ...
spotify:track:1Eolhana7nKHYpcYpdVcT5 ...
spotify:track:2AxCeJ6PSsBYiTckM0HLY7 ...
spotify:track:0NWPxcsf5vdjdiFUI8NgkP ...
spotify:track:3qebSLgUwaEqbwCM5sj5Kh ...
spotify:track:5H6Jp0syB5yEPk7SWYdlmk ...
spotify:track:23MrkN7g6Q5U7GLIxNHN1B ...
spotify:track:1ntxpzIUbSsizvuAy6lTYY ...
spotify:track:5L

...

spotify:track:0WSeqoAtxc3vmvupvACRSK ...
spotify:track:3rGaIIWHrBMPI9fLQpx0FP ...
spotify:track:2NvnhJgS9q36tY7PwRLyYO ...
spotify:track:5AhbaEkadQ4TPRZnEk6xF4 ...
spotify:track:472a4lSuWfcFCGSSyls2NO ...
spotify:track:3Mz2nuJLr0z9vLjo6Jk2I2 ...
spotify:track:0dSe5nhc5CUq3HHbQgja5E ...
spotify:track:56ZPG0x5xqdAroBbfCNBQy ...
spotify:track:3jBVKhBcUi1HwOyl1MgEXa ...
spotify:track:4Cd2bWO4CWo0RbisnWZrH8 ...
spotify:track:1ycypFzMcG0IucW0od7Ap2 ...
spotify:track:0WmkPM2RN7gm17wMVB7vST ...
spotify:track:37HLDnFm5Vbt24Ccjc3wnA ...
spotify:track:75BGwASipV5oNrbUgtGIhi ...
spotify:track:2Eu7vVd3HnuOVjVl57hG66 ...
spotify:track:1sJjZw7KZ3WSEfdKeTAVs4 ...
spotify:track:6cwMOwwDzvkWV1ZgTr3cGI ...
spotify:track:1CBVMonVjYdtx7aoGAHCEl ...
spotify:track:5mKlBA4ib3GGpbRl19YNpo ...
spotify:track:4IN34Iq5T9x1vN6MASxvS9 ...
spotify:track:50mwGp3PgKoZldhBvfy2cf ...
spotify:track:0r4FEHukFd50r43uTuYmHY ...
spotify:track:4KyJQKAFvEH4F52qSMuwIF ...
spotify:track:2Ef5Vgw0ShuKGoNfO6Nrof ...
spotify:track:75

spotify:track:2sxjbw2YMiVQlDVktiynd7 ...
spotify:track:2q2CRCN29nmqveoOrWKSHv ...
spotify:track:6W8rlIepxKPBFsmpM2zO76 ...
spotify:track:7L34Is2rWf408IlHmbju79 ...
spotify:track:47fEXwCTjSCyntdxVBWj44 ...
spotify:track:1xHNCP1x9ZyIi3HidWqaIv ...
spotify:track:09BtpgxNENGEDVXDorANsx ...
spotify:track:0VV1bDcbXWQbGBE2wxwgQx ...
spotify:track:4pNDusZhcM2Cl0gMtSzGgn ...
spotify:track:2x3hj1Y13NXv2cKHck0B9B ...
spotify:track:4ZSjlaPRNrYEeym7fJMywk ...
spotify:track:47Kxj6HoFSUUBnVw6tpZQB ...
spotify:track:3iyZG1EB9G3LqPVXVwFSfV ...
spotify:track:60mmTochpWHDCRe7SNewuI ...
spotify:track:2YsvobYVLbcVOsJ4NgZtgC ...
spotify:track:24ifWf75zkTyZ7fPMWl6iD ...
spotify:track:3o9ZHqWpwychTEhB7pee9T ...
spotify:track:07sbJawa4WXRLVerJaT83i ...
spotify:track:2jGRfdqBvMyRgoBU7dxHOg ...
spotify:track:15Ihw9YYcsvarDutay5dGQ ...
spotify:track:5oD8tVXfPOojReq4wxEja9 ...
spotify:track:0Uq0Xm2gkNPrN5Td37dOSD ...
spotify:track:3dSQT6f7Tv1h2sqJszJaum ...
spotify:track:7rkhPVYJFvcIleTFSuy2W8 ...
spotify:track:0S

spotify:track:6eo04yKCNmMegAvdQPEhm8 ...
spotify:track:4kTCYsbseIEd7aJmeCIP2n ...
spotify:track:0kRpRDf4otbnR7NlUNyye0 ...
spotify:track:0JyuEt2VTlBvGMBrM1XJ5x ...
spotify:track:5QoyjtDJbhzZxvKfdyTbV0 ...
spotify:track:7nepk7xMc9PqMDZRpjRFj9 ...
spotify:track:4k1vZn3n2PLxEvyU50ePEi ...
spotify:track:6YPIY1f39d5yjemPjkdAaN ...
spotify:track:0MOpmEFLzFdhLhOYVLBbHn ...
spotify:track:7aLeI6sFEEnBwbvf91RXXd ...
spotify:track:6eJ2QB5o3CoMeCXrO9ALcx ...
spotify:track:5B2wtdNoYyYurkJzh30GAl ...
spotify:track:1ckFfmW82nvvT7mav3qZhN ...
spotify:track:1BVtoyZtDlmuogw6FmL9DE ...
spotify:track:2TScNFVGQ6PQS9cGLMyULI ...
spotify:track:2qKUG1Q09xLoXXZkXsJDPj ...
spotify:track:4FFGa0VCjKmwVjGebvOqVR ...
spotify:track:0Ib7ZfLQBHxIzwYb0tsQOh ...
spotify:track:725DEO0ypwfs4RdV3efGIE ...
spotify:track:1qJxkZIdZCbQFdOjCiOjKc ...
spotify:track:79XqDhfNEMTVx8E7wZbr53 ...
spotify:track:7mxbWOkGGGqIlGvX6j6OQt ...
spotify:track:0uHzHWC18T9aQZ3F7q8XR7 ...
spotify:track:0y3DRFq950ExwuRMgv8Hqs ...
spotify:track:53

spotify:track:2wtrNtNgK53xoNCGyNyvhI ...
spotify:track:5ei1dtRoiyUScGJz6nTXk6 ...
spotify:track:29nsi4msngAG3TI0Biqpq5 ...
spotify:track:1pGAw7MGjukjiPRLtzZCwY ...
spotify:track:3U0k17CTuK35GSZo6ofClL ...
spotify:track:3ePoh7vNGKOTuzjMNOB7I7 ...
spotify:track:5rcwVk85FLHsELX96A1AJF ...
spotify:track:2SnJmKJAp8fPJ9WV5eqs6g ...
spotify:track:7LnsgTP6qHkgyZTQXrd7Kr ...
spotify:track:1CbcqIf9ijjlqhApdaKSTj ...
spotify:track:4QfrzvemlAWITHFZQBy2db ...
spotify:track:1c8VPiBfqUMFwwnlx9q9gv ...
spotify:track:2gwMiMAQO5httSgTrTfePY ...
spotify:track:3WxzMN4tMdyPotUYolFRa3 ...
spotify:track:4wiFIIiD0ffT3eKADwQMXX ...
spotify:track:7G7tgVYORlDuVprcYHuFJh ...
spotify:track:6tXnRSvuNgOq4QcxpIN54r ...
spotify:track:0eDwzWuy2gf1RJzqWl0dkF ...
spotify:track:4ZYCwq1EMPY5zZt195ARWg ...
spotify:track:1Yc2k9b9PNZSB6P5S53SPC ...
spotify:track:1fiDe8sb2DMEU4JGPOIEB1 ...
spotify:track:4oh6NKtMdiXIhgGGQBR09b ...
spotify:track:7sDHD9tCtnPqGNTRHy0kSR ...
spotify:track:1oSPStIOjKzNhPqCU1xXVi ...
spotify:track:2B

spotify:track:3HbnrHREdJO8VUvoLNnSRf ...
spotify:track:6h5w5jYYA6xL8dENWs8ZDt ...
spotify:track:5P6IezyndUp33FBt7yM1nf ...
spotify:track:1ziIG2TBQ1MKGhmyURP7a1 ...
spotify:track:6vU0nFKGKVLUeqp7Gj4QuE ...
spotify:track:2dj6TorhKDlNMcbWlMQmoR ...
spotify:track:1WVMXowGnVTm3glTOWM7U3 ...
spotify:track:0YyF4r6l79hu2VlfIaRxqx ...
spotify:track:3JAQIeQ3EX0BmMqvLNO4nM ...
spotify:track:0iBhYQiU2DMM7iB6bfDvCE ...
spotify:track:7JR4dimzmJQ6Iz9kVvHLOg ...
spotify:track:7phMyl0FZBv3WQkDgeLodt ...
spotify:track:4hrgnUUPyL6pEgPlw2GggF ...
spotify:track:2irLLFQfPp7oaVDUKNxGvs ...
spotify:track:0m20pALKNQRr7liC8NRSQk ...
spotify:track:4ZCLM74MYhUy80TPiaAuee ...
spotify:track:0MOi5oWv5Y0Ea9p7BdXepF ...
spotify:track:63IItP7504rQwYgZqmsznL ...
spotify:track:2pH562uKymLxYYrLiHJelf ...
spotify:track:4C8qZRxrteBY9F8r2oZrV3 ...
spotify:track:2QfjV18pg4Ye7Vo4TsSobq ...
spotify:track:4g2TNvnLKj3UPeuoJEZ1gM ...
spotify:track:57DRDn3OKZyKQniRNAjYIQ ...
spotify:track:7uyWFXp842IIc6GzFIUDvA ...
spotify:track:1d

In [47]:
ae.df_songs.to_csv('../raw_data/songs.csv')

In [55]:
ae.df_analysis.to_csv('../raw_data/analysis.csv')

In [63]:
ae.df_features.to_csv('../raw_data/features.csv')